In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("nlp_test_data_final.csv")

In [3]:
df = df.drop(["course_organization","course_students_enrolled","Unnamed: 0"], axis=1,errors= "ignore")

In [4]:
df.head(5)

,course_title,course_Certificate_type,course_rating,course_difficulty,Prerequisites
0,(ISC)² Systems Security Certified Practitioner...,SPECIALIZATION,4.7,Beginner,"Basic IT Knowledge, Networking Fundamentals, C..."
1,A Crash Course in Causality: Inferring Causal...,COURSE,4.7,Intermediate,"Statistics, Probability Theory, Research Metho..."
2,A Crash Course in Data Science,COURSE,4.5,Mixed,"Python, Linear Algebra, Basic Machine Learning"
3,A Law Student's Toolkit,COURSE,4.7,Mixed,"Analytical Thinking, Basic Legal Knowledge, Re..."
4,A Life of Happiness and Fulfillment,COURSE,4.8,Mixed,"Interest in Psychology, Emotional Awareness, O..."


In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["course_difficulty"] = le.fit_transform(df["course_difficulty"])

In [6]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download("punkt_tab")
nltk.download("stopwords")
ps = PorterStemmer()
def transform_text(text):
    text = text.lower()
    text = nltk.word_tokenize(text)
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)
    text = y[:]
    y.clear()
    for i in text:
        if i not in stopwords.words("english") and i not in string.punctuation:
            y.append(i)
    text = y[:]
    y.clear()
    for i in text:
      y.append(ps.stem(i))
    return " ".join(y)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
def collect_user_profile():
  user_profile = {
      "skills":input("ENTER YOUR CURRENT SKILLS:"),
      "interests":input("ENTER YOUR INTERESTS:"),
      "GOAL":input("ENTER WHAT DO YOU WANT TO BECOME:")
  }
  return user_profile
user_profile = " ".join(collect_user_profile().values())
user_profile = transform_text(user_profile)

ENTER YOUR CURRENT SKILLS:python
ENTER YOUR INTERESTS:machine learning, deep learning
ENTER WHAT DO YOU WANT TO BECOME:Data Scientist


In [8]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def get_sbert_embeddings(text):
  return sbert_model.encode(text,convert_to_tensor=True)

user_embeddings = get_sbert_embeddings(user_profile)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
df["description"] = df["course_title"] + df["Prerequisites"]
for i in range(len(df)):
  df["description"][i] = transform_text(df["description"][i])

Streaming output truncated to the last 5000 lines.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["description"][i] = transform_text(df["description"][i])
<ipython-input-9-35220d4fce2a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["description"][i] = transform_text(df["description"][i])
<ipython-input-9-35220d4fce2a>:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will be

In [10]:
import torch
import torch.nn as nn
def normalize(embeddings:torch.Tensor) -> torch.Tensor:
  if embeddings.dim() == 1:
    norm = torch.norm(embeddings,p=2,dim=0,keepdim=True)
    return embeddings/(norm + 1e-8)
  elif embeddings.dim() == 2:
      norm = torch.norm(embeddings,p=2,dim=1,keepdim=True)
      return embeddings/(norm + 1e-8)


In [11]:
normalized_embeddings = normalize(user_embeddings)


In [12]:
!pip install faiss-cpu
import faiss
import numpy as np
course_embeddings_list = []
user_embedding = user_embeddings.detach().cpu().numpy().astype("float32")
for i in range(len(df)):
  x = df.loc[i,"description"]
  x = get_sbert_embeddings(x)
  x = x.detach().cpu().numpy().astype("float32")
  course_embeddings_list.append(x)

course_embeddings_np = np.vstack(course_embeddings_list)
if user_embedding.ndim == 1:
  user_embedding = np.expand_dims(user_embeddings,axis = 0)
embedding_dim = course_embeddings_np.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(course_embeddings_np)
k = 5
distances ,indices = index.search(user_embedding,k)
cosine_similarities = 1-(np.square(distances)/2)
print(cosine_similarities)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 17.6 MB/s eta 0:00:00
[[0.752041   0.72568166 0.6997839  0.68548155 0.68122023]]


In [13]:
print('Top Similar Courses:')
for idx,sim in zip(indices[0],cosine_similarities[0]):
  course_title = df.loc[idx,"course_title"]
  course_rating = df.loc[idx,"course_rating"]
  print(f"Course Title: {course_title}")
  print(f"Course Rating: {course_rating}")

Top Similar Courses:
Course Title: IBM AI Engineering
Course Rating: 4.6
Course Title: Introduction to Artificial Intelligence (AI)
Course Rating: 4.7
Course Title: AI For Everyone
Course Rating: 4.8
Course Title: Mountains 101
Course Rating: 4.9
Course Title: AI for Medicine
Course Rating: 4.7
